# 🎬 End-to-End Machine Learning: Box Office Prediction Pipeline
## **Part 1: Data Acquisition & Feature Engineering**

**Project:** Predicting the Opening Week Revenue for ***Avatar: Fire and Ash*** (Dec 2025).

**Author:** Arunima Chakraborty

**Date:** December 2025

---

### **Notebook Objective**
This notebook serves as the **Data Engineering Engine** for the project. It connects to the **TMDb API**, scrapes 20+ years of high-revenue movie data, and engineers complex features (Star Power, Competition Scores, Inflation Adjustment) to prepare a dataset for the regression models in Part 2.

### **Pipeline Overview**
1.  **Phase 0: Setup** - Library installation and API authentication.
2.  **Phase 1: Target Acquisition** - Scrapes 20,000 movies using time-chunking to bypass API limits.
3.  **Phase 2: Metadata Enrichment** - Queries granular details (Budget, Cast, Keywords) and constructs the target variable proxy.
4.  **Phase 3: Feature Engineering** - Applies historical tracking for Star Power and implements inflation adjustment (CPI).

---

In [2]:
#hide warnings
import warnings
warnings.filterwarnings('ignore')

# 1. Install Libraries
!pip install tmdbv3api cpi pandas numpy scikit-learn seaborn matplotlib shap xgboost requests beautifulsoup4 fake-useragent

# 2. Imports
import pandas as pd
import numpy as np
import cpi
import time
import ast
import itertools
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
from tmdbv3api import TMDb, Find, Movie, Discover
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
from fake_useragent import UserAgent

# 3. User Configuration
API_KEY = "2408ba3105fdbab4b16bb458113ed558"  # <--- PASTE YOUR KEY HERE
START_YEAR = 2000
END_YEAR = 2025
CURRENT_YEAR = 2025

# 4. Setup API
tmdb = TMDb()
tmdb.api_key = API_KEY
tmdb_movie = Movie()
discover = Discover()

# 5. Suppress CPI Warnings
import logging
logging.getLogger("cpi").setLevel(logging.ERROR)
try: cpi.update()
except: pass

print("Phase 0: Setup Complete.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.8 MB/s eta 0:00:00


Phase 0: Setup Complete.


## **PHASE 1: Target Acquisition**
**Goal:** Scrape 20,000 high-revenue movies using Time Chunking to bypass API limits.

In [3]:
print("Phase 1: Target Acquisition (Fresh Start)...")

movies_fallback = []
target_count = 20000

# Split into chunks to bypass 10k limit
time_chunks = [
    (START_YEAR, START_YEAR + 12),      # 2000-2012
    (START_YEAR + 13, END_YEAR)         # 2013-2025
]

for start_chunk, end_chunk in time_chunks:
    print(f"Fetching movies for years {start_chunk}-{end_chunk}...")
    page = 1

    while page <= 500 and len(movies_fallback) < target_count:
        try:
            disc = discover.discover_movies({
                'sort_by': 'revenue.desc',
                'primary_release_date.gte': f'{start_chunk}-01-01',
                'primary_release_date.lte': f'{end_chunk}-12-31',
                'with_original_language': 'en',
                'page': page
            })

            if not disc: break

            for m in disc:
                if not any(x['tmdb_id'] == m['id'] for x in movies_fallback):
                    movies_fallback.append({
                        'tmdb_id': m['id'],
                        'release_date': m.get('release_date')
                    })

            if len(movies_fallback) % 1000 == 0:
                print(f"  Progress: {len(movies_fallback)} movies found...")

            page += 1
            time.sleep(0.1)
        except: break

target_df = pd.DataFrame(movies_fallback)
target_df.to_csv('phase1_target_list.csv', index=False)
print(f"\nPhase 1 Complete. {len(target_df)} movies ready.")

Phase 1: Target Acquisition (Fresh Start)...
Fetching movies for years 2000-2012...
  Progress: 1000 movies found...
  Progress: 2000 movies found...
  Progress: 3000 movies found...
Fetching movies for years 2013-2025...

Phase 1 Complete. 19170 movies ready.


## **Part 2: Feature Engineering**
**Goal:** Fetch Metadata.

**Fix:** Saves tmdb_title so we can see movie names later.

In this phase, we iterate through our filtered movie list and query the **The Movie Database (TMDb) API** for granular metadata. This step addresses the "Sparse Data" problem by engineering high-value features and normalizing financial metrics.

**Key Methodological Actions:**
1.  **Target Variable Construction (The "Proxy"):**
    * *Problem:* TMDb provides "Total Global Revenue" but rarely splits out "Domestic Opening Week."
    * *Solution:* We engineered a proxy target using industry-standard decay curves:
        `Target = Global Revenue × 0.40 (Domestic Share) × 0.35 (First Week Share)`.
    * *Justification:* This standardizes the target variable, allowing the model to learn opening-week dynamics even when explicit data is missing.

2.  **Economic Normalization (CPI Adjustment):**
    * We integrated the **CPI (Consumer Price Index)** library to adjust all historical Budgets and Revenues to **2025 Dollars**. This ensures that a \$100M budget in 2009 (*Avatar*) is mathematically comparable to a \$400M budget in 2025.

3.  **Noise Filtering:**
    * We implemented a "Garbage Filter" to drop entries with `< $1,000` in budget or revenue, removing student films and data entry errors that could skew the regression gradients.

4.  **Metadata Extraction:**
    * **MPAA Rating:** Encoded as ordinal data (G=0 to NC-17=4).
    * **Talent IDs:** Extracted Director and Top 3 Cast IDs to build the "Star Power" history in Phase 3.
    * **Franchise Status:** Boolean flag for sequels (`belongs_to_collection`).

In [4]:
print("Phase 2: Feature Engineering...")

final_data = []
skip_low_data = 0

def get_mpaa(releases):
    if not hasattr(releases, 'results'): return 2
    for r in releases.results:
        if r['iso_3166_1'] == 'US':
            for d in r['release_dates']:
                if d['certification']:
                    return {'G':0, 'PG':1, 'PG-13':2, 'R':3, 'NC-17':4}.get(d['certification'], 2)
    return 2

# Iterate through target list
for i, row in target_df.iterrows():
    try:
        m_id = row['tmdb_id']
        details = tmdb_movie.details(m_id, append_to_response='credits,keywords,release_dates')

        budget = getattr(details, 'budget', 0)
        revenue = getattr(details, 'revenue', 0)

        # Filter "Garbage" (Low Budget/Revenue)
        if budget < 1000 or revenue < 1000:
            skip_low_data += 1
            continue

        # Target Calculation (Proxy for Opening Weekend)
        # Global * 40% (Domestic Share) * 35% (First Week Share)
        target_val = revenue * 0.40 * 0.35

        # Feature Extraction
        keywords = [k['name'].lower() for k in list(details.keywords.keywords)] if hasattr(details, 'keywords') else []

        top_cast = []
        director = None
        if hasattr(details, 'credits'):
            if hasattr(details.credits, 'cast'):
                top_cast = [c['id'] for c in list(details.credits.cast)[:3]]
            if hasattr(details.credits, 'crew'):
                director = next((c['id'] for c in list(details.credits.crew) if c['job'] == 'Director'), None)

        is_sequel = 1 if hasattr(details, 'belongs_to_collection') and details.belongs_to_collection else 0
        coll_id = details.belongs_to_collection['id'] if is_sequel else None

        # Inflation Adjustment
        year = pd.to_datetime(details.release_date).year
        try: inf_factor = cpi.inflate(1.0, year, to=CURRENT_YEAR)
        except: inf_factor = 1.0

        final_data.append({
            'tmdb_title': details.title,
            'release_date': pd.to_datetime(details.release_date),
            'Budget_Adj': budget * inf_factor,
            'Domestic_First_Week_Adj': target_val * inf_factor,
            'Runtime': getattr(details, 'runtime', 0),
            'MPAA_Rating': get_mpaa(getattr(details, 'release_dates', None)),
            'Director_ID': director,
            'Top_Cast_IDs': top_cast,
            'Keywords': keywords,
            'Is_Sequel': is_sequel,
            'Collection_ID': coll_id,
            'Distributor_List': list(getattr(details, 'production_companies', []))
        })

    except: pass

    # Progress & Rate Limiting
    if i % 100 == 0:
        print(f"Processed {i}/{len(target_df)}... (Saved: {len(final_data)})")
    time.sleep(0.08)

df = pd.DataFrame(final_data)
df.to_csv('phase2_features.csv', index=False)
print(f"Phase 2 Complete. {len(df)} movies saved.")

Phase 2: Feature Engineering...
Processed 0/19170... (Saved: 1)
Processed 100/19170... (Saved: 101)
Processed 200/19170... (Saved: 201)
Processed 300/19170... (Saved: 301)
Processed 400/19170... (Saved: 401)
Processed 500/19170... (Saved: 500)
Processed 600/19170... (Saved: 600)
Processed 700/19170... (Saved: 699)
Processed 800/19170... (Saved: 798)
Processed 900/19170... (Saved: 897)
Processed 1000/19170... (Saved: 995)
Processed 1100/19170... (Saved: 1091)
Processed 1200/19170... (Saved: 1189)
Processed 1300/19170... (Saved: 1284)
Processed 1400/19170... (Saved: 1382)
Processed 1500/19170... (Saved: 1478)
Processed 1600/19170... (Saved: 1573)
Processed 1700/19170... (Saved: 1663)
Processed 1800/19170... (Saved: 1752)
Processed 1900/19170... (Saved: 1840)
Processed 2000/19170... (Saved: 1925)
Processed 2100/19170... (Saved: 2008)
Processed 2200/19170... (Saved: 2088)
Processed 2300/19170... (Saved: 2164)
Processed 2500/19170... (Saved: 2301)
Processed 2800/19170... (Saved: 2486)
Proce

In the following phase, we transform the raw metadata into quantifiable features suitable for machine learning. This involves both static processing and dynamic historical tracking.

**Key Engineering Steps:**
1.  **Distributor Tiering:** We categorize production companies into Tiers (1=Major Studios like Disney/Warner, 2=Mid-Majors, 3=Indies) to serve as a proxy for marketing power and distribution reach.
2.  **Theater Count Imputation:** Since historical theater counts are often missing, we estimate them based on the Distributor Tier (e.g., Tier 1 releases typically secure ~4,000+ screens).
3.  **Genre One-Hot Encoding:** We explicitly flag high-value genres (Sci-Fi, Adventure, Action) and premium formats (IMAX/3D) derived from keyword tags.
4.  **"Time-Travel Safe" History:** We implement a rolling loop that iterates through movies chronologically to calculate cumulative statistics without data leakage:
    * **Star Power:** Sum of inflation-adjusted gross revenue generated by the director and top cast in *previous* movies.
    * **Actor Familiarity:** A graph-based metric counting how often the cast members have worked together before.
    * **Competition Score:** A count of other major releases launching within a ±7 day window.
    * **Sequel Performance:** Retrieves the adjusted gross of the immediate predecessor if the film is part of a franchise.

In [5]:
print("Phase 3: Cleaning & History...")

if 'df' in locals() and not df.empty:
    df = df.sort_values('release_date').reset_index(drop=True)

    # 1. Distributor Processing
    def get_dist_info(comps):
        names = []
        if isinstance(comps, list):
            for c in comps:
                if hasattr(c, 'name'): names.append(c.name)
                elif isinstance(c, dict) and 'name' in c: names.append(c['name'])

        primary = names[0] if names else "Unknown"
        tier = 3
        lower = [n.lower() for n in names]
        if any(k in n for n in lower for k in ['disney', 'warner', 'universal', 'paramount', 'sony', 'marvel', 'pixar']): tier = 1
        elif any(k in n for n in lower for k in ['lionsgate', 'a24', 'netflix', 'dreamworks']): tier = 2
        return pd.Series([primary, tier])

    df[['Distributor_Name', 'Distributor_Tier']] = df['Distributor_List'].apply(get_dist_info)

    # 2. Impute Theater Count
    def estimate_screens(row):
        if row['Distributor_Tier'] == 1: return 4000
        if row['Distributor_Tier'] == 2: return 2500
        return 500
    df['Theater_Count'] = df.apply(estimate_screens, axis=1)

    # 3. Enhanced Genre Engineering
    def check_genre(k_list, targets):
        if isinstance(k_list, list): return 1 if any(t in k_list for t in targets) else 0
        return 0

    df['Genre_SciFi'] = df['Keywords'].apply(lambda x: check_genre(x, ['sci-fi', 'science fiction', 'alien', 'future', 'space']))
    df['Genre_Adventure'] = df['Keywords'].apply(lambda x: check_genre(x, ['adventure', 'journey', 'quest']))
    df['Genre_Action'] = df['Keywords'].apply(lambda x: check_genre(x, ['action', 'war', 'battle', 'fight']))
    df['Genre_Fantasy'] = df['Keywords'].apply(lambda x: check_genre(x, ['fantasy', 'magic', 'myth', 'creature', 'monster']))
    df['Has_IMAX_3D'] = df['Keywords'].apply(lambda x: check_genre(x, ['3d', 'imax']))
    df['Release_Month'] = df['release_date'].dt.month

    # 4. Rolling History Loop (Time Travel Safe)
    dir_hist, cast_hist, collab_graph = {}, {}, {}
    c_dir, c_cast, c_fam, c_prev, c_comp = [], [], [], [], []

    for idx, row in df.iterrows():
        # Competition Score
        start, end = row['release_date'] - timedelta(days=7), row['release_date'] + timedelta(days=7)
        c_comp.append(max(0, df[(df['release_date'] >= start) & (df['release_date'] <= end)].shape[0] - 1))

        # Sequel History
        prev = 0
        if row['Is_Sequel'] and row['Collection_ID']:
            sibs = df[(df['Collection_ID'] == row['Collection_ID']) & (df['release_date'] < row['release_date'])]
            if not sibs.empty: prev = sibs.iloc[-1]['Domestic_First_Week_Adj']
        c_prev.append(prev)

        # Star Power
        c_dir.append(dir_hist.get(row['Director_ID'], 0))
        c_cast.append(sum([cast_hist.get(a, 0) for a in row['Top_Cast_IDs']]))

        # Familiarity
        fam = 0
        if len(row['Top_Cast_IDs']) > 1:
            for pair in itertools.combinations(row['Top_Cast_IDs'], 2):
                fam += collab_graph.get(frozenset(pair), 0)
        c_fam.append(fam)

        # Update Trackers
        rev = row['Domestic_First_Week_Adj']
        if row['Director_ID']: dir_hist[row['Director_ID']] = dir_hist.get(row['Director_ID'], 0) + rev
        for a in row['Top_Cast_IDs']: cast_hist[a] = cast_hist.get(a, 0) + rev
        if len(row['Top_Cast_IDs']) > 1:
            for pair in itertools.combinations(row['Top_Cast_IDs'], 2):
                key = frozenset(pair)
                collab_graph[key] = collab_graph.get(key, 0) + 1

    df['Competition_Score'] = c_comp
    df['Prev_Movie_Gross_Adj'] = c_prev
    df['Director_Prev_Gross'] = c_dir
    df['Cast_Star_Power'] = c_cast
    df['Actor_Familiarity'] = c_fam

    # 5. Imputation & Save
    mask = (df['Is_Sequel'] == 1) & (df['Prev_Movie_Gross_Adj'] == 0)
    if mask.sum() > 0:
        ratio = (df['Prev_Movie_Gross_Adj'] / df['Budget_Adj']).median()
        df.loc[mask, 'Prev_Movie_Gross_Adj'] = df.loc[mask, 'Budget_Adj'] * (ratio if not np.isnan(ratio) else 3.0)

    df.to_csv('Avatar_Final_Dataset_Enhanced.csv', index=False)
    print("Phase 3 Complete. Dataset Saved.")

Phase 3: Cleaning & History...
Phase 3 Complete. Dataset Saved.


In [13]:
# ==========================================
# DOWNLOAD THE DATASET
# ==========================================
from google.colab import files

try:
    print("Downloading 'Avatar_Final_Dataset_Enhanced.csv'...")
    files.download('Avatar_Final_Dataset_Enhanced.csv')
except ImportError:
    print("Error: This download code only works in Google Colab.")
    print("If you are running locally, the file is already in your folder.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
**Status:** Data Pipeline Complete.

**Next Step:** Proceeding to Notebook `02_EDA_Modeling_and_Final_Prediction.ipynb` for EDA, Modeling, and the Final Prediction.